### Imports

In [79]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision

In [80]:
from torchsummary import summary

### Set Device

In [81]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Hyperparameters

In [82]:
in_channel = 3
num_classes = 10
learning_rate = 1e4
batch_size = 1024
num_epochs = 5

### Load pretrained-model & modify it

https://pytorch.org/vision/stable/models.html

In [83]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    
    def forward(self, x):
        return x

In [84]:
model = torchvision.models.vgg16(pretrained = True)

In [104]:
model.avgpool = Identity()
model.classifier = nn.Sequential(
    nn.Linear(512, 100),
    nn.ReLU(),
    nn.Linear(100,10)
)
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [105]:
summary(model,(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
              ReLU-2           [-1, 64, 32, 32]               0
            Conv2d-3           [-1, 64, 32, 32]          36,928
              ReLU-4           [-1, 64, 32, 32]               0
         MaxPool2d-5           [-1, 64, 16, 16]               0
            Conv2d-6          [-1, 128, 16, 16]          73,856
              ReLU-7          [-1, 128, 16, 16]               0
            Conv2d-8          [-1, 128, 16, 16]         147,584
              ReLU-9          [-1, 128, 16, 16]               0
        MaxPool2d-10            [-1, 128, 8, 8]               0
           Conv2d-11            [-1, 256, 8, 8]         295,168
             ReLU-12            [-1, 256, 8, 8]               0
           Conv2d-13            [-1, 256, 8, 8]         590,080
             ReLU-14            [-1, 25

### Load Data

In [106]:
train_dataset = datasets.CIFAR10(root='datasets/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.CIFAR10(root='datasets/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [107]:
train_dataset[0][0].shape

torch.Size([3, 32, 32])

### Loss and Optimizer

In [108]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Check Accuracy

In [109]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad(): ## when check accuracy, no need to check gradients
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            
            scores = model(x) # 64 x 10 dim
            _, predictions = scores.max(1) # axis = 1

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0) # 64
        
        accuracy = float(num_correct) / float(num_samples)*100
    
    return accuracy

### Train Model

In [110]:
def train_model(model, train_loader, test_loader, num_epochs=10, print_every = 1):
    model.train()
    
    for epoch in range(num_epochs):
        for batch_idx, (data, targets) in enumerate(train_loader):
                
            # Forwardprop
            scores = model(data)
            loss = loss_fn(scores, targets)

            # Backwardprop
            optimizer.zero_grad()
            loss.backward()

            # Gradient Descent - Adam Optimization
            optimizer.step()
        
        # Accuracy
        print(f'Epoch {epoch+1}| Test Accuracy: {check_accuracy(test_loader, model):.2f}% | Train Accuracy: {check_accuracy(train_loader, model):.2f}%')
#       

In [111]:
train_model(model, train_loader, test_loader)

KeyboardInterrupt: 

In [112]:
check_accuracy(test_loader, model)

10.0